### Tools

**Models** can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code.
**Tools** are pairing of:

1. A Schema including the name of the tool, a description, and/or argument definations(often a JSON schema).
2. A function or coroutine to execute.